This Query is created by user sohamjcpp

In [2]:
%% sql
WITH Premia_Pcom_customer AS (
  SELECT 
    cust_code AS intermediary_code,
    cust_name AS intermediary_name,
    cust_name AS agent_mobile_no,
    cust_name AS agent_email_id
  FROM Premia
),
Bancs_T_RPDB_POL_PARTY AS (
  SELECT 
    party_code AS intermediary_code,
    party_name AS intermediary_name
  FROM Bancs
),
Intermediary AS (
  SELECT 
    p.intermediary_code,
    p.intermediary_name,
    p.agent_mobile_no,
    p.agent_email_id,
    m.intermediary_type
  FROM Premia_Pcom_customer p
  LEFT JOIN Bancs_T_RPDB_POL_PARTY m ON SUBSTR(p.intermediary_code, 1, 2) = SUBSTR(m.cust_code, 1, 2)
),
Derived AS (
  SELECT 
    CASE 
      WHEN (CURRENT_DATE - l.aml_acknowledged_timestamp) >= 365 THEN 'Yes' 
      WHEN (CURRENT_DATE - l.aml_acknowledged_timestamp) < 365 THEN 'No' 
    END AS due_aml_training,
    DATE_ADD(CURRENT_DATE, INTERVAL -1 DAY) AS reported_date,
    CURRENT_TIMESTAMP() AS load_timestamp
  FROM vo_login l
),
Final_Result AS (
  SELECT 
    i.intermediary_code,
    i.intermediary_name,
    i.intermediary_type,
    i.agent_mobile_no,
    i.agent_email_id,
    l.due_aml_training,
    l.reported_date,
    l.load_timestamp,
    m.fufiller_code,
    m.fulfiller_name,
    m.office_code,
    m.office_name,
    m.zone_name,
    m.mkt_zone_name
  FROM Intermediary i
  LEFT JOIN agg_policy_master_report_daily m ON i.intermediary_code = m.intermediary_code
  LEFT JOIN Derived l ON 1 = 1
)
SELECT 
  f.intermediary_code,
  f.intermediary_name,
  f.intermediary_type,
  f.agent_mobile_no,
  f.agent_email_id,
  f.due_aml_training,
  f.reported_date,
  f.load_timestamp,
  f.fufiller_code,
  f.fulfiller_name,
  f.office_code,
  f.office_name,
  f.zone_name,
  f.mkt_zone_name
FROM Final_Result f;